In [ ]:
import plotly
# data analysis and wrangling
import pandas_profiling
import pandas as pd
import numpy as np
import random as rnd
import xgboost as xgb
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, RobustScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer

In [ ]:
from plotly.offline import download_plotlyjs , init_notebook_mode , plot , iplot
import plotly.graph_objs as go
import plotly.express as px
init_notebook_mode(connected = True)

In [ ]:
data = pd.read_csv('/content/train_qnU1GcL.csv')

test = pd.read_csv(r"/content/test_rec.csv")


df = data.copy()

df.head(5)

df = df.drop(['id'] , axis = 1)

In [ ]:
df['uwscore'] = df['uwscore'].fillna(df['uwscore'].mean())


In [ ]:
df.fillna(0 , inplace = True)

In [ ]:
tst  = test.copy()

In [ ]:
tst = tst.drop(['id'] , axis = 1)

In [ ]:
tst['uwscore'] = tst['uwscore'].fillna(tst['uwscore'].mean())



In [ ]:
tst.late2.fillna(1 , inplace = True)

In [ ]:
tst.fillna(0 , inplace = True)

In [ ]:
tst

In [ ]:
df['age'] = df['age'].div(365)

In [ ]:
tst['age'] = tst['age'].div(365)

In [ ]:
df['Income'] = np.log(df['Income'])

In [ ]:
tst['Income'] = np.log(tst['Income'])

In [ ]:
#df.profile_report()

# Performing Mean Encoding

In [ ]:
pd.crosstab(index = df['channel'] , columns =df['target'])

In [ ]:
means = df.groupby('channel').target.mean()

In [ ]:
df['channel'+'mean_target'] = df['channel'].map(means)

In [ ]:
tst['channel'+'mean_target'] = tst['channel'].map(means)

In [ ]:
df

# NEW feature

In [ ]:


num_late_months = df.late1 * 4.5 + df.late2 * 9 + df.late3 * 12

df['late_months'] = num_late_months

In [ ]:
num_late_months = tst.late1 * 4.5 + tst.late2 * 9 + tst.late3 * 12

tst['late_months'] = num_late_months

In [ ]:
In_npp = df['Income'].div(df['nppaid'])
df['In_npp'] = In_npp

In [ ]:
In_npp = tst['Income'].div(tst['nppaid'])
tst['In_npp'] = In_npp

# New Feature Underscore Age

In [ ]:
uw_age = df['uwscore'].div(df['age'])
df['uw_age'] = uw_age

In [ ]:
uw_age = tst['uwscore'].div(tst['age'])
tst['uw_age'] = uw_age

# New Feature for total no. of premiums for policy

In [ ]:
np_pp = []
for i in range(0 ,len(df.ppaid)):
    if df.iloc[i , 0] == 0:
        np_pp.append(0)
    else:
        l =  (df.iloc[i , 7])/(df.iloc[i,0])
        np_pp.append(l)
        

In [ ]:
np_pp

In [ ]:
df['np_pp'] = np_pp

In [ ]:
df

In [ ]:
np_pp = []
for i in range(0 ,len(tst.ppaid)):
    if tst.iloc[i , 0] == 0:
        np_pp.append(0)
    else:
        l =  (tst.iloc[i , 7])/(tst.iloc[i,0])
        np_pp.append(l)

In [ ]:
tst

In [ ]:
tst['np_pp'] = np_pp

In [ ]:
#df = df[['index','ppaid','age','Income','late1','late2','late3','uwscore','nppaid','late_months','In_npp','uw_age','np_pp','channel','residence_area_type','target']]

# New feature Income acc Age

In [ ]:
In_Age = df.Income.div(df.age)

In [ ]:
df['In_Age'] = In_Age

In [ ]:
tst['In_Age'] = tst.Income.div(tst.age)

In [ ]:
df

In [ ]:
tst

# Income late_months

In [ ]:
In_late = df.late_months.div(df.Income)

In [ ]:
df['In_late'] = In_late

In [ ]:
tst['In_late'] = tst.late_months.div(tst.Income)

# Scaling robust scaler on features

In [ ]:
tst

In [ ]:
std_scaler = StandardScaler()
rob_scaler = RobustScaler()
minmax = MinMaxScaler()

# ROBSCALER

In [ ]:
df['ppaid'] = rob_scaler.fit_transform(df['ppaid'].values.reshape(-1,1))
df['age'] = rob_scaler.fit_transform(df['age'].values.reshape(-1,1))
df['Income'] = rob_scaler.fit_transform(df['Income'].values.reshape(-1,1))
df['late1'] = minmax.fit_transform(df['late1'].values.reshape(-1,1))
df['late2'] = minmax.fit_transform(df['late2'].values.reshape(-1,1))
df['late3'] = minmax.fit_transform(df['late3'].values.reshape(-1,1))
df['uwscore'] = rob_scaler.fit_transform(df['uwscore'].values.reshape(-1,1))
df['nppaid'] = rob_scaler.fit_transform(df['nppaid'].values.reshape(-1,1))
df['late_months'] = minmax.fit_transform(df['late_months'].values.reshape(-1,1))
df['In_npp'] = rob_scaler.fit_transform(df['In_npp'].values.reshape(-1,1))
df['uw_age'] = rob_scaler.fit_transform(df['uw_age'].values.reshape(-1,1))
df['np_pp'] = rob_scaler.fit_transform(df['np_pp'].values.reshape(-1,1))
df['In_Age'] = rob_scaler.fit_transform(df['In_Age'].values.reshape(-1,1))
df['In_late'] = rob_scaler.fit_transform(df['In_late'].values.reshape(-1,1))

In [ ]:
tst['ppaid'] = rob_scaler.fit_transform(tst['ppaid'].values.reshape(-1,1))
tst['age'] = rob_scaler.fit_transform(tst['age'].values.reshape(-1,1))
tst['Income'] = rob_scaler.fit_transform(tst['Income'].values.reshape(-1,1))
tst['late1'] = minmax.fit_transform(tst['late1'].values.reshape(-1,1))
tst['late2'] = minmax.fit_transform(tst['late2'].values.reshape(-1,1))
tst['late3'] = minmax.fit_transform(tst['late3'].values.reshape(-1,1))
tst['uwscore'] = rob_scaler.fit_transform(tst['uwscore'].values.reshape(-1,1))
tst['nppaid'] = rob_scaler.fit_transform(tst['nppaid'].values.reshape(-1,1))
tst['late_months'] = minmax.fit_transform(tst['late_months'].values.reshape(-1,1))
tst['In_npp'] = rob_scaler.fit_transform(tst['In_npp'].values.reshape(-1,1))
tst['uw_age'] = rob_scaler.fit_transform(tst['uw_age'].values.reshape(-1,1))
tst['np_pp'] = rob_scaler.fit_transform(tst['np_pp'].values.reshape(-1,1))
tst['In_Age'] = rob_scaler.fit_transform(tst['In_Age'].values.reshape(-1,1))
tst['In_late'] = rob_scaler.fit_transform(tst['In_late'].values.reshape(-1,1))

In [ ]:
df.head(2)

In [ ]:
tst.head(2)

# MINMAX SCALER

In [ ]:
df['uwscore'] = minmax.fit_transform(df['uwscore'].values.reshape(-1,1))
df['nppaid'] = minmax.fit_transform(df['nppaid'].values.reshape(-1,1))
df['late_months'] = minmax.fit_transform(df['late_months'].values.reshape(-1,1))
df['age'] = minmax.fit_transform(df['age'].values.reshape(-1,1))
df['Income'] = minmax.fit_transform(df['Income'].values.reshape(-1,1))
df['ppaid'] = minmax.fit_transform(df['ppaid'].values.reshape(-1,1))
df['late1'] = minmax.fit_transform(df['late1'].values.reshape(-1,1))
df['late2'] = minmax.fit_transform(df['late2'].values.reshape(-1,1))
df['late3'] = minmax.fit_transform(df['late3'].values.reshape(-1,1))
df['In_npp'] =minmax.fit_transform(df['In_npp'].values.reshape(-1,1))

In [ ]:
tst['uwscore'] = minmax.fit_transform(tst['uwscore'].values.reshape(-1,1))
tst['nppaid'] = minmax.fit_transform(tst['nppaid'].values.reshape(-1,1))
tst['late_months'] = minmax.fit_transform(tst['late_months'].values.reshape(-1,1))
tst['age'] = minmax.fit_transform(tst['age'].values.reshape(-1,1))
tst['Income'] = minmax.fit_transform(tst['Income'].values.reshape(-1,1))
tst['ppaid'] = minmax.fit_transform(tst['ppaid'].values.reshape(-1,1))
tst['late1'] = minmax.fit_transform(tst['late1'].values.reshape(-1,1))
tst['late2'] = minmax.fit_transform(tst['late2'].values.reshape(-1,1))
tst['late3'] = minmax.fit_transform(tst['late3'].values.reshape(-1,1))
tst['In_npp'] =minmax.fit_transform(tst['In_npp'].values.reshape(-1,1))

In [ ]:
#tst['late1_1'] = minmax.fit_transform(tst['late1'].values.reshape(-1,1))
#tst['late2_1'] = minmax.fit_transform(tst['late2'].values.reshape(-1,1))
#tst['late3_1'] = minmax.fit_transform(tst['late3'].values.reshape(-1,1))
#tst['In_npp'] = minmax.fit_transform(tst['In_npp'].values.reshape(-1,1))

# Std Scaler

In [ ]:
tst['ppaid'] = std_scaler.fit_transform(tst['ppaid'].values.reshape(-1,1))
tst['age'] = std_scaler.fit_transform(tst['age'].values.reshape(-1,1))
tst['Income'] = std_scaler.fit_transform(tst['Income'].values.reshape(-1,1))
tst['late1'] = minmax.fit_transform(tst['late1'].values.reshape(-1,1))
tst['late2'] =  minmax.fit_transform(tst['late2'].values.reshape(-1,1))
tst['late3'] =  minmax.fit_transform(tst['late3'].values.reshape(-1,1))
tst['uwscore'] =  std_scaler.fit_transform(tst['uwscore'].values.reshape(-1,1))
tst['nppaid'] =  std_scaler.fit_transform(tst['nppaid'].values.reshape(-1,1))
tst['late_months'] =  minmax.fit_transform(tst['late_months'].values.reshape(-1,1))
tst['In_npp'] =  std_scaler.fit_transform(tst['In_npp'].values.reshape(-1,1))
tst['uw_age'] =  std_scaler.fit_transform(tst['uw_age'].values.reshape(-1,1))
tst['np_pp'] =  std_scaler.fit_transform(tst['np_pp'].values.reshape(-1,1))
tst['In_late'] = std_scaler.fit_transform(tst['In_late'].values.reshape(-1,1))

In [ ]:
df['ppaid'] = std_scaler.fit_transform(df['ppaid'].values.reshape(-1,1))
df['age'] = std_scaler.fit_transform(df['age'].values.reshape(-1,1))
df['Income'] = std_scaler.fit_transform(df['Income'].values.reshape(-1,1))
df['late1'] = std_scaler.fit_transform(df['late1'].values.reshape(-1,1))
df['late2'] =  std_scaler.fit_transform(df['late2'].values.reshape(-1,1))
df['late3'] =  std_scaler.fit_transform(df['late3'].values.reshape(-1,1))
df['uwscore'] =  std_scaler.fit_transform(df['uwscore'].values.reshape(-1,1))
df['nppaid'] =  std_scaler.fit_transform(df['nppaid'].values.reshape(-1,1))
df['late_months'] =  std_scaler.fit_transform(df['late_months'].values.reshape(-1,1))
df['In_npp'] =  std_scaler.fit_transform(df['In_npp'].values.reshape(-1,1))
df['uw_age'] =  std_scaler.fit_transform(df['uw_age'].values.reshape(-1,1))
df['np_pp'] =  std_scaler.fit_transform(df['np_pp'].values.reshape(-1,1))
df['In_late'] = std_scaler.fit_transform(df['In_late'].values.reshape(-1,1))

In [ ]:
tst.describe()

In [ ]:
df.drop(['ppaid','age','Income','late1','late2','late3','uwscore','nppaid','late_months'], axis=1, inplace=True)

In [ ]:
tst.drop(['ppaid','age','Income','late1','late2','late3','uwscore','nppaid','late_months'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
tst

In [ ]:
df = df[['ppaid_1','age_1','Income_1','late1_1','late2_1','late3_1','uwscore_1','nppaid_1','late_months_1','In_npp','uw_age','np_pp','channel','residence_area_type','target']]

In [ ]:
tst = tst[['ppaid_1','age_1','Income_1','late1_1','late2_1','late3_1','uwscore_1','nppaid_1','late_months_1','In_npp','uw_age','np_pp','channel','residence_area_type']]

In [ ]:
df 

In [ ]:
tst

In [ ]:
df = df.drop('index' , axis = 1)

# performing test train splitting

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

In [ ]:
sss =  StratifiedShuffleSplit(n_splits=5, random_state=None, test_size=0.33)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

In [ ]:
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain))
print(test_counts_label/ len(original_ytest))

In [ ]:
original_Xtrain

In [ ]:
original_Xtest

In [ ]:
tst

# Lable Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
enco = LabelEncoder()

original_Xtrain['residence_area_type']  = np.array(enco.fit_transform(original_Xtrain['residence_area_type']))


In [ ]:
#original_Xtest['channel']  = np.array(enco.fit_transform(original_Xtest['channel']))
original_Xtest['residence_area_type']  = np.array(enco.fit_transform(original_Xtest['residence_area_type']))

In [ ]:
#tst['channel']  = np.array(enco.fit_transform(tst['channel']))
tst['residence_area_type']  = np.array(enco.fit_transform(tst['residence_area_type']))

In [ ]:
original_Xtrain['channel']  = np.array(enco.fit_transform(original_Xtrain['channel']))
original_Xtest['channel']  = np.array(enco.fit_transform(original_Xtest['channel']))
tst['channel']  = np.array(enco.fit_transform(tst['channel']))

In [ ]:
original_Xtest.channel.value_counts()

# Using Dummy Variables

In [ ]:
original_Xtrain = pd.get_dummies(original_Xtrain)
original_Xtest = pd.get_dummies(original_Xtest)

In [ ]:
tst = pd.get_dummies(tst)

In [ ]:
tst

In [ ]:
#original_Xtrain

In [ ]:
original_ytrain

In [ ]:
original_Xtrain.drop([ 'residence_area_type_Urban','channel_E' ], axis=1, inplace=True)
original_Xtest.drop([ 'residence_area_type_Urban','channel_E' ], axis=1, inplace=True)


In [ ]:
tst.drop([ 'residence_area_type_Urban','channel_E'  ], axis=1, inplace=True)

In [ ]:
original_Xtest

# XGBOOST MERI JAAAN

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(original_Xtrain,label= original_ytrain)
dtest = xgb.DMatrix(original_Xtest, label= original_ytest)

In [ ]:
params = {
    # Parameters that we are going to tune.
 'learning_rate' : 0.1,
 'n_estimators' : 1000,
 'max_depth' : 5,
 'min_child_weight' :1,
 'gamma' : 0,
 'subsample':0.8,
 'colsample_bytree':0.8,
 'objective' :'binary:logistic',
 'nthread' : 4,
 'scale_pos_weight' : 1,
 'seed': 27
}

In [ ]:
params['eval_metric'] = "auc"

In [ ]:
num_boost_round = 999

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=100
    
)
xgb.plot_importance(model)

In [ ]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'auc'},
    early_stopping_rounds=50
)
cv_results

In [ ]:
cv_results['test-auc-mean'].max()

# Grid_Search CV for optimization of base model

In [ ]:
params = {
    # Parameters that we are going to tune.
 'learning_rate' : 0.1,
 'n_estimators' : 83,
 'max_depth' : 5,
 'min_child_weight' :1,
 'gamma' : 0,
 'subsample':0.8,
 'colsample_bytree':0.8,
 'objective' :'binary:logistic',
 'nthread' : 4,
 'scale_pos_weight' : 1,
 'seed': 27
}

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=45, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(original_Xtrain, original_ytrain)
gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[1,2,3]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=45, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(original_Xtrain, original_ytrain)
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test2b = {
 'min_child_weight':[2,3,4]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=45, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(original_Xtrain, original_ytrain)
gsearch2b.best_params_, gsearch2b.best_score_

Tunnning GAMMA

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=45, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(original_Xtrain, original_ytrain)
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
params = {
    # Parameters that we are going to tune.
 'learning_rate' : 0.1,
 'n_estimators' : 1000,
 'max_depth' : 5,
 'eval_metric' : 'auc',
 'min_child_weight' :4,
 'gamma' : 0,
 'subsample':0.8,
 'colsample_bytree':0.8,
 'objective' :'binary:logistic',
 'nthread' : 4,
 'scale_pos_weight' : 1,
 'seed': 27
}

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=10000,
    evals=[(dtest, "Test")],
    early_stopping_rounds=100
    
)
xgb.plot_importance(model)

Tune subsample and colsample_bytree

In [ ]:
param_test4 = {
 'subsample':[i/10.0 for i in range(2,10)],
 'colsample_bytree':[i/10.0 for i in range(2,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=45, max_depth=5,
 min_child_weight= 1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(original_Xtrain, original_ytrain)
gsearch4.best_params_, gsearch4.best_score_

In [ ]:
param_test5 = {
 'subsample':[i/100.0 for i in range(65,85,5)],
 'colsample_bytree':[i/100.0 for i in range(85,95,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=45, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(original_Xtrain, original_ytrain)
gsearch5.best_params_, gsearch5.best_score_

 Tuning Regularization Parameters

In [ ]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=45, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(original_Xtrain, original_ytrain)
gsearch6.best_params_, gsearch6.best_score_

In [ ]:
# Better Performance (Best Model)
param_test7 = {
 'reg_alpha':[0.001,0.005,0.01,0.1,0.5] # Furter comes out to be 0.01
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=57, max_depth=4,
 min_child_weight=4, gamma=0.2, subsample=0.65, colsample_bytree=0.85,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(original_Xtrain, original_ytrain)
gsearch7.best_params_, gsearch7.best_score_

In [ ]:
params = {
 # Parameters that we are going to tune.
 'learning_rate' : 0.005,
 'n_estimators' : 1000,
 'max_depth' : 4,
 'eval_metric' : 'auc',
 'min_child_weight' :4,
 'gamma' : 0.2,
 'subsample':0.65,
 'colsample_bytree':0.85,
  'reg_alpha':0.01,
 'objective' :'binary:logistic',
 'nthread' : 4,
 'scale_pos_weight' : 1,
 'seed': 27
}

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=4000,
    evals=[(dtest, "Test")],
    early_stopping_rounds=100
    
)
xgb.plot_importance(model)

In [ ]:
params['max_depth'] = 6
params['min_child_weight'] = 9

In [ ]:
# Tunning for sub_sample and Colsample
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [ ]:
max_auc = float("Inf")
best_params = None
for subsample, colsample in gridsearch_params:
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # Update our parameters
    params['subsample'] = subsample
    params['colsample'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=50
    )
    # Update best MAE
    mean_auc = cv_results['test-auc-mean'].max()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
    if mean_auc < max_auc:
        max_auc = mean_auc
        best_params = (subsample,colsample)
print("Best params: {}, {}, AUC: {}".format(best_params[0], best_params[1], max_auc))

In [ ]:
params['subsample'] = 0.7
params['colsample_bytree'] = 0.7

In [ ]:
max_auc = float("Inf")
best_params = None
for eta in [.05, .01, .005,0.001]:
    print("CV with eta={}".format(eta))
    # Update our parameters
    params['eta'] = eta
    
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=50
    )
    # Update best MAE
    mean_auc = cv_results['test-auc-mean'].max()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
    if mean_auc < max_auc:
        max_auc = mean_auc
        best_params = (eta)
print("Best params: {}, {}, AUC: {}".format(best_params, max_auc))

In [ ]:
params['eta'] = 0.005

In [ ]:
params
{'colsample_bytree': 0.5,
 'eta': 0.001,
 'eval_metric': 'auc',
 'max_depth': 15,
 'min_child_weight': 3,
 'gamma': 6,
 'n_estimator': 280,
 'objective': 'binary:logistic',
 'subsample': 0.5}

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=50
)

# SMOTE APPlication

In [ ]:
#sm = SMOTE(random_state = 12 )
#X_train , y_train = sm.fit_sample(original_Xtrain, original_ytrain)



In [ ]:
#print("After OverSampling, counts of label '1': {}".format(sum(y_train==1)))
#print("After OverSampling, counts of label '0': {}".format(sum(y_train==0)))

# LightGBM

In [ ]:
import lightgbm



#categorical_features = ['channel','residence_area_type']

train_data = lightgbm.Dataset(original_Xtrain,label= original_ytrain)
test_data = lightgbm.Dataset(original_Xtest, label= original_ytest)


parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.001,
    'verbose': 0,
    'max_bin' : 300
    
}
evals_results = {}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       evals_result=evals_results,
                       num_boost_round=5000,
                       early_stopping_rounds=100)

In [ ]:
y_pred = model.predict(original_Xtest)

print(roc_auc_score(original_ytest, y_pred))

# Predicting tst 

In [ ]:
y_pred = model.predict(tst)

# End Conversion to csv file for Submission

In [ ]:
dc = pd.DataFrame(test , columns = ['id'])

dc['target'] = y_pred

dc

dc.to_csv ('D:/datasc challange/INsurance problem/finalfew.csv', index = False, header=True)